<a href="https://colab.research.google.com/github/elifabanoz/bone-fracture-classification/blob/main/seng445_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Course: Computer Vision
#Date: 20.12.2025

In [ ]:
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Scikit-learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Configuration
IMG_SIZE = 224  # EfficientNetB0 default input size
BATCH_SIZE = 8  # Small batch size for small dataset
EPOCHS = 50
N_FOLDS = 10

In [ ]:
# Data paths (Google Colab structure - same level as sample_data)
DATA_DIR = "data"
FRACTURE_DIR = os.path.join(DATA_DIR, "fracture")
NORMAL_DIR = os.path.join(DATA_DIR, "normal")

In [ ]:
def load_and_preprocess_image(image_path, img_size=IMG_SIZE):
    """Load and preprocess a single image."""
    img = load_img(image_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    return img_array

In [ ]:
def load_dataset():
    """Load all images and labels from the dataset."""
    images = []
    labels = []

    # Load fracture images (label = 1)
    print("Loading fracture images...")
    if os.path.exists(FRACTURE_DIR):
        fracture_files = [f for f in os.listdir(FRACTURE_DIR)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        for filename in fracture_files:
            filepath = os.path.join(FRACTURE_DIR, filename)
            try:
                img = load_and_preprocess_image(filepath)
                images.append(img)
                labels.append(1)  # Fracture = 1
            except Exception as e:
                print(f"Error loading {filepath}: {e}")
        print(f"  Loaded {len([l for l in labels if l == 1])} fracture images")
    else:
        raise FileNotFoundError(f"Fracture directory not found: {FRACTURE_DIR}")

    # Load normal images (label = 0)
    print("Loading normal images...")
    if os.path.exists(NORMAL_DIR):
        normal_files = [f for f in os.listdir(NORMAL_DIR)
                        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        for filename in normal_files:
            filepath = os.path.join(NORMAL_DIR, filename)
            try:
                img = load_and_preprocess_image(filepath)
                images.append(img)
                labels.append(0)  # Normal = 0
            except Exception as e:
                print(f"Error loading {filepath}: {e}")
        print(f"  Loaded {len([l for l in labels if l == 0])} normal images")
    else:
        raise FileNotFoundError(f"Normal directory not found: {NORMAL_DIR}")

    X = np.array(images, dtype=np.float32)
    y = np.array(labels, dtype=np.int32)

    print(f"\nDataset Summary:")
    print(f"  Total images: {len(X)}")
    print(f"  Fracture: {np.sum(y == 1)}, Normal: {np.sum(y == 0)}")
    print(f"  Image shape: {X[0].shape}")

    return X, y

In [ ]:
def get_data_augmentation():
    """Create data augmentation generator for training."""
    return ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=False,
        zoom_range=0.15,
        shear_range=0.1,
        fill_mode='nearest'
    )

In [ ]:
def get_validation_generator():
    """Create generator for validation (only preprocessing, no augmentation)."""
    return ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
    )

In [ ]:
def create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    """
    Create transfer learning model using EfficientNetB0.

    Architecture:
    - EfficientNetB0 pretrained on ImageNet as feature extractor
    - Global Average Pooling
    - Dense layers with dropout for regularization
    - Sigmoid output for binary classification
    """
    # Load pretrained EfficientNetB0
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Build the model
    inputs = keras.Input(shape=input_shape)

    # Base model (feature extraction)
    x = base_model(inputs, training=False)

    # Classification head
    x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = layers.BatchNormalization(name='bn1')(x)
    x = layers.Dropout(0.4, name='dropout1')(x)
    x = layers.Dense(256, activation='relu', name='dense1')(x)
    x = layers.BatchNormalization(name='bn2')(x)
    x = layers.Dropout(0.3, name='dropout2')(x)
    x = layers.Dense(64, activation='relu', name='dense2')(x)
    x = layers.Dropout(0.2, name='dropout3')(x)
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs, outputs, name='FractureClassifier')

    return model, base_model

In [ ]:

def train_fold(X_train, y_train, X_val, y_val, fold_num):
    """
    Train model for a single fold with two-phase training:
    Phase 1: Train only classification head (frozen base)
    Phase 2: Fine-tune top layers of base model
    """
    print(f"\n{'='*60}")
    print(f"FOLD {fold_num}/{N_FOLDS}")
    print(f"{'='*60}")
    print(f"Train: {len(X_train)} samples | Val: {len(X_val)} samples")
    print(f"Train distribution - Fracture: {np.sum(y_train==1)}, Normal: {np.sum(y_train==0)}")

    # Clear any previous session
    keras.backend.clear_session()

    # Create model
    model, base_model = create_model()

    # Freeze base model for phase 1
    base_model.trainable = False

    # Data generators
    train_datagen = get_data_augmentation()
    val_datagen = get_validation_generator()

    # Callbacks
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=8,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    )

    # ==================== PHASE 1: Train Classification Head ====================
    print("\n[Phase 1] Training classification head (base frozen)...")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Fit phase 1
    train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=SEED)
    val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, shuffle=False)

    steps_per_epoch = max(1, len(X_train) // BATCH_SIZE)
    validation_steps = max(1, len(X_val) // BATCH_SIZE)

    model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=15,
        validation_data=val_generator,
        validation_steps=validation_steps,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )

    # ==================== PHASE 2: Fine-tune Top Layers ====================
    print("\n[Phase 2] Fine-tuning top layers of base model...")

    # Unfreeze top layers of base model
    base_model.trainable = True

    # Freeze early layers, unfreeze last 30 layers
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    # Recompile with lower learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # New callbacks for fine-tuning
    early_stop_ft = EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr_ft = ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        min_lr=1e-8,
        verbose=1
    )

    # Fit phase 2
    train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=SEED)
    val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, shuffle=False)

    model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=validation_steps,
        callbacks=[early_stop_ft, reduce_lr_ft],
        verbose=1
    )

    # ==================== Evaluation ====================
    print("\nEvaluating on validation set...")

    # Preprocess validation data
    X_val_processed = tf.keras.applications.efficientnet.preprocess_input(X_val.copy())

    # Predict
    y_pred_proba = model.predict(X_val_processed, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()

    # Calculate accuracy
    fold_accuracy = accuracy_score(y_val, y_pred)

    print(f"\n>>> Fold {fold_num} Accuracy: {fold_accuracy:.4f} <<<")

    # Cleanup
    del model, base_model
    keras.backend.clear_session()

    return y_pred, y_val, fold_accuracy

In [ ]:
def run_10fold_cross_validation(X, y):
    """
    Run 10-fold stratified cross-validation.

    Note: StratifiedKFold ensures each fold has similar class distribution.
    This is important for imbalanced datasets.
    """
    print("\n" + "="*60)
    print("STARTING 10-FOLD STRATIFIED CROSS-VALIDATION")
    print("="*60)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    fold_accuracies = []
    all_y_true = []
    all_y_pred = []

    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y), start=1):
        # Split data for this fold
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Train and evaluate
        y_pred, y_true, accuracy = train_fold(X_train, y_train, X_val, y_val, fold_idx)

        # Store results
        fold_accuracies.append(accuracy)
        all_y_true.extend(y_true)
        all_y_pred.extend(y_pred)

    return fold_accuracies, np.array(all_y_true), np.array(all_y_pred)

In [ ]:
def print_final_results(fold_accuracies, y_true, y_pred):
    """Print comprehensive final results."""

    print("\n" + "="*60)
    print("FINAL RESULTS - 10-FOLD CROSS-VALIDATION")
    print("="*60)

    # Individual fold results
    print("\nAccuracy per fold:")
    for i, acc in enumerate(fold_accuracies, 1):
        status = "✓" if acc >= 0.95 else "○" if acc >= 0.85 else "✗"
        print(f"  Fold {i:2d}: {acc:.4f} {status}")

    # Summary statistics
    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    min_acc = np.min(fold_accuracies)
    max_acc = np.max(fold_accuracies)

    print(f"\n{'─'*40}")
    print(f"Mean Accuracy:  {mean_acc:.4f}")
    print(f"Std Deviation:  {std_acc:.4f}")
    print(f"Min Accuracy:   {min_acc:.4f}")
    print(f"Max Accuracy:   {max_acc:.4f}")
    print(f"{'─'*40}")

    # Classification report
    print("\nOverall Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Normal', 'Fracture'], digits=4))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(f"                  Predicted")
    print(f"                Normal  Fracture")
    print(f"Actual Normal     {cm[0,0]:4d}     {cm[0,1]:4d}")
    print(f"       Fracture   {cm[1,0]:4d}     {cm[1,1]:4d}")

    # Grading
    print("\n" + "="*60)
    print("GRADE CALCULATION")
    print("="*60)
    print(f"\n10-Fold CV Mean Accuracy: {mean_acc:.4f}")

    if mean_acc >= 0.95:
        grade = 15
        status = "FULL GRADE"
    elif mean_acc >= 0.85:
        grade = 10
        status = "PARTIAL GRADE"
    else:
        grade = 5
        status = "MINIMUM GRADE"

    print(f"Grade: {grade}/15 points ({status})")

    if mean_acc >= 0.95:
        print("✓ Accuracy >= 0.95: Requirement met!")
    elif mean_acc >= 0.85:
        print("○ Accuracy >= 0.85 but < 0.95")
    else:
        print("✗ Accuracy < 0.85")

    return mean_acc

In [ ]:
def main():
    """Main execution function."""

    print("="*60)
    print("BONE FRACTURE CLASSIFICATION SYSTEM")
    print("Transfer Learning with EfficientNetB0")
    print("="*60)

    # Check GPU
    print("\n[1/4] Checking hardware...")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"✓ GPU available: {gpus[0]}")
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(f"  GPU config warning: {e}")
    else:
        print("✗ No GPU found - using CPU (will be slower)")

    # Verify directories
    print("\n[2/4] Verifying data directories...")
    print(f"  Looking for: {DATA_DIR}/")
    print(f"               ├── fracture/")
    print(f"               └── normal/")

    if not os.path.exists(DATA_DIR):
        raise FileNotFoundError(
            f"\nERROR: '{DATA_DIR}' folder not found!\n"
            f"Make sure 'data' folder is at the same level as 'sample_data'"
        )
    if not os.path.exists(FRACTURE_DIR):
        raise FileNotFoundError(f"\nERROR: '{FRACTURE_DIR}' folder not found!")
    if not os.path.exists(NORMAL_DIR):
        raise FileNotFoundError(f"\nERROR: '{NORMAL_DIR}' folder not found!")

    print("✓ All directories found")

    # Load data
    print("\n[3/4] Loading dataset...")
    X, y = load_dataset()

    # Run cross-validation
    print("\n[4/4] Running 10-fold cross-validation...")
    fold_accuracies, y_true, y_pred = run_10fold_cross_validation(X, y)

    # Print results
    mean_accuracy = print_final_results(fold_accuracies, y_true, y_pred)

    print("\n" + "="*60)
    print("EXECUTION COMPLETED")
    print("="*60)

    return mean_accuracy

In [ ]:
# Entry point
if __name__ == "__main__":
    final_accuracy = main()

BONE FRACTURE CLASSIFICATION SYSTEM
Transfer Learning with EfficientNetB0

[1/4] Checking hardware...
✗ No GPU found - using CPU (will be slower)

[2/4] Verifying data directories...
  Looking for: data/
               ├── fracture/
               └── normal/
✓ All directories found

[3/4] Loading dataset...
Loading fracture images...
  Loaded 111 fracture images
Loading normal images...
  Loaded 82 normal images

Dataset Summary:
  Total images: 193
  Fracture: 111, Normal: 82
  Image shape: (224, 224, 3)

[4/4] Running 10-fold cross-validation...

STARTING 10-FOLD STRATIFIED CROSS-VALIDATION

FOLD 1/10
Train: 173 samples | Val: 20 samples
Train distribution - Fracture: 99, Normal: 74
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

[Phase 1] Training classification head (base frozen)...
Epoch 1/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 32s 872ms/step - accuracy: 0.6579 - loss: 0.7465 - val_accuracy: 0.8750 - val_loss: 0.3767 - learning_rate: 0.0010
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44m


>>> Fold 5 Accuracy: 1.0000 <<<

FOLD 6/10
Train: 174 samples | Val: 19 samples
Train distribution - Fracture: 100, Normal: 74

[Phase 1] Training classification head (base frozen)...
Epoch 1/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 29s 754ms/step - accuracy: 0.6692 - loss: 0.6052 - val_accuracy: 0.8750 - val_loss: 0.4790 - learning_rate: 0.0010
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.8750 - loss: 0.2058 - val_accuracy: 0.8750 - val_loss: 0.4705 - learning_rate: 0.0010
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 556ms/step - accuracy: 0.8890 - loss: 0.2086 - val_accuracy: 0.9375 - val_loss: 0.3369 - learning_rate: 0.0010
Epoch 4/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 1.0000 - loss: 0.1337 - val_accuracy: 0.9375 - val_loss: 0.3319 - learning_rate: 0.0010
Epoch 5/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 991ms/step - accuracy: 0.8900 - loss: 0.2060 - val_accuracy: 0.8750 - val_loss: 0.2820 - learning_rate: 0.0010
Epoch 6/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - a


>>> Fold 6 Accuracy: 1.0000 <<<

FOLD 7/10
Train: 174 samples | Val: 19 samples
Train distribution - Fracture: 100, Normal: 74

[Phase 1] Training classification head (base frozen)...
Epoch 1/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 33s 784ms/step - accuracy: 0.5590 - loss: 0.9389 - val_accuracy: 0.9375 - val_loss: 0.4738 - learning_rate: 0.0010
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.8750 - loss: 0.2131 - val_accuracy: 0.9375 - val_loss: 0.4620 - learning_rate: 0.0010
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 607ms/step - accuracy: 0.8281 - loss: 0.3792 - val_accuracy: 1.0000 - val_loss: 0.2811 - learning_rate: 0.0010
Epoch 4/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8750 - loss: 0.6260 - val_accuracy: 1.0000 - val_loss: 0.2782 - learning_rate: 0.0010
Epoch 5/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8983 - loss: 0.2229 - val_accuracy: 0.9375 - val_loss: 0.2600 - learning_rate: 0.0010
Epoch 6/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accu